# Очистка данных, Предобработка данных, Feature Engineering

Импорт библиотек:

In [1]:
# from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# для обработки текста
import re, inflect, spacy

# настрока отображения вывода
import warnings
warnings.filterwarnings("ignore")

Загрузка датасета с информацие о произведениях искусства:

In [2]:
artw_df = pd.read_csv('artworks.csv')
artw_df.head()

,Artwork ID,Title,Artist ID,Name,Date,Medium,Dimensions,Acquisition Date,Credit,Catalogue,...,Classification,Object Number,Diameter (cm),Circumference (cm),Height (cm),Length (cm),Width (cm),Depth (cm),Weight (kg),Duration (s)
0,2,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",6210,Otto Wagner,1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",1996-04-09,Fractional and promised gift of Jo Carole and ...,Y,...,Architecture,885.1996,NaN,NaN,48.6000,NaN,168.9000,NaN,NaN,NaN
1,3,"City of Music, National Superior Conservatory ...",7470,Christian de Portzamparc,1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",1995-01-17,Gift of the architect in honor of Lily Auchinc...,Y,...,Architecture,1.1995,NaN,NaN,40.6401,NaN,29.8451,NaN,NaN,NaN
2,4,"Villa near Vienna Project, Outside Vienna, Aus...",7605,Emil Hoppe,1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",1997-01-15,Gift of Jo Carole and Ronald S. Lauder,Y,...,Architecture,1.1997,NaN,NaN,34.3000,NaN,31.8000,NaN,NaN,NaN
3,5,"The Manhattan Transcripts Project, New York, N...",7056,Bernard Tschumi,1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",1995-01-17,Purchase and partial gift of the architect in ...,Y,...,Architecture,2.1995,NaN,NaN,50.8000,NaN,50.8000,NaN,NaN,NaN
4,6,"Villa, project, outside Vienna, Austria, Exter...",7605,Emil Hoppe,1903,"Graphite, color pencil, ink, and gouache on tr...","15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",1997-01-15,Gift of Jo Carole and Ronald S. Lauder,Y,...,Architecture,2.1997,NaN,NaN,38.4000,NaN,19.1000,NaN,NaN,NaN


Загрузка датасета с информацией о художниках:

In [3]:
artists_df = pd.read_csv('artists.csv')
artists_df.head()

,Artist ID,Name,Nationality,Gender,Birth Year,Death Year
0,1,Robert Arneson,American,Male,1930.0,1992.0
1,2,Doroteo Arnaiz,Spanish,Male,1936.0,NaN
2,3,Bill Arnold,American,Male,1941.0,NaN
3,4,Charles Arnoldi,American,Male,1946.0,NaN
4,5,Per Arnoldi,Danish,Male,1941.0,NaN


Визуализация общей информации о датасетах:

In [4]:
display(artw_df.info(), artists_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130262 entries, 0 to 130261
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Artwork ID          130262 non-null  int64  
 1   Title               130200 non-null  object 
 2   Artist ID           128802 non-null  object 
 3   Name                128802 non-null  object 
 4   Date                127950 non-null  object 
 5   Medium              118343 non-null  object 
 6   Dimensions          118799 non-null  object 
 7   Acquisition Date    124799 non-null  object 
 8   Credit              127192 non-null  object 
 9   Catalogue           130262 non-null  object 
 10  Department          130262 non-null  object 
 11  Classification      130262 non-null  object 
 12  Object Number       130262 non-null  object 
 13  Diameter (cm)       1399 non-null    float64
 14  Circumference (cm)  10 non-null      float64
 15  Height (cm)         111893 non-nul

None

None

Приведение названий колонок к релевантному типу

In [5]:
def col_name_transformer(column: str) -> str:
    trash_symbols = "()"
    new_name = column.lower()
    for sym in trash_symbols:
        new_name = new_name.replace(sym, '')
    new_name = new_name.replace(' ', '_')
    return new_name

artw_df.columns = [*map(col_name_transformer, artw_df.columns)]
artists_df.columns = [*map(col_name_transformer, artists_df.columns)]

display(artw_df.info(), artists_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130262 entries, 0 to 130261
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artwork_id        130262 non-null  int64  
 1   title             130200 non-null  object 
 2   artist_id         128802 non-null  object 
 3   name              128802 non-null  object 
 4   date              127950 non-null  object 
 5   medium            118343 non-null  object 
 6   dimensions        118799 non-null  object 
 7   acquisition_date  124799 non-null  object 
 8   credit            127192 non-null  object 
 9   catalogue         130262 non-null  object 
 10  department        130262 non-null  object 
 11  classification    130262 non-null  object 
 12  object_number     130262 non-null  object 
 13  diameter_cm       1399 non-null    float64
 14  circumference_cm  10 non-null      float64
 15  height_cm         111893 non-null  float64
 16  length_cm         73

None

None

Визуализация описательной статистики численных и категориальных признаков по 2 датасетам:

In [6]:
display(artw_df.describe(),
        artw_df.describe(include='object'),
        artists_df.describe(),
        artists_df.describe(include='object'))

,artwork_id,diameter_cm,circumference_cm,height_cm,length_cm,width_cm,depth_cm,weight_kg,duration_s
count,130262.000000,1399.000000,10.000000,111893.000000,736.000000,111003.000000,11443.000000,298.000000,3.084000e+03
mean,82501.371636,23.248939,44.868020,37.712992,89.117417,38.176838,18.291359,1248.278691,7.830060e+03
std,58124.331702,45.460079,28.631604,48.151347,329.717487,67.250118,57.703925,11856.456824,1.185046e+05
min,2.000000,0.635000,9.900000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000e+00
25%,34171.250000,7.900000,23.500000,18.100000,17.031875,17.800000,0.000000,5.443200,2.100000e+02
50%,69541.500000,13.700000,36.000000,27.940056,26.700000,25.400100,0.700000,19.051100,7.200000e+02
75%,128955.750000,24.782500,71.125000,44.450100,79.100000,44.800000,13.335013,65.318000,3.613250e+03
max,218011.000000,914.400000,83.800000,9140.000000,8321.056600,9144.000000,1808.483617,185067.585957,6.283065e+06


,title,artist_id,name,date,medium,dimensions,acquisition_date,credit,catalogue,department,classification,object_number
count,130200,128802,128802,127950,118343,118799,124799,127192,130262,130262,130262,130262
unique,94881,13575,13530,9373,19348,78464,1876,7030,2,9,28,130262
top,Untitled,229,Eugène Atget,1971,Gelatin silver print,"Approx. 8 11/16 x 7 1/16"" (22 x 18 cm)",1964-10-06,The Louis E. Stern Collection,Y,Prints & Illustrated Books,Print,885.1996
freq,7180,5050,5050,1748,14103,923,11220,10927,71333,60128,30807,1


,artist_id,birth_year,death_year
count,15091.000000,11237.000000,4579.000000
mean,18297.556027,1930.852719,1974.287399
std,16632.963898,34.531997,31.153665
min,1.000000,1730.000000,1795.000000
25%,4195.500000,1910.000000,1958.000000
50%,8593.000000,1936.000000,1980.000000
75%,33088.500000,1956.000000,1998.000000
max,67695.000000,2012.000000,2017.000000


,name,nationality,gender
count,15091,12603,12019
unique,15039,125,3
top,Unknown Designer,American,Male
freq,24,5198,9820


Удаление лишних колонок:

In [7]:
artw_df.drop(['artwork_id', 'artist_id', 'acquisition_date', 'catalogue', 'object_number', 'credit'], axis=1, inplace=True)
artists_df.drop(['artist_id'], axis=1, inplace=True)

display(artw_df.head(), artists_df.head())

,title,name,date,medium,dimensions,department,classification,diameter_cm,circumference_cm,height_cm,length_cm,width_cm,depth_cm,weight_kg,duration_s
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",Architecture & Design,Architecture,NaN,NaN,48.6000,NaN,168.9000,NaN,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",Architecture & Design,Architecture,NaN,NaN,40.6401,NaN,29.8451,NaN,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",Architecture & Design,Architecture,NaN,NaN,34.3000,NaN,31.8000,NaN,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",Architecture & Design,Architecture,NaN,NaN,50.8000,NaN,50.8000,NaN,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,1903,"Graphite, color pencil, ink, and gouache on tr...","15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",Architecture & Design,Architecture,NaN,NaN,38.4000,NaN,19.1000,NaN,NaN,NaN


,name,nationality,gender,birth_year,death_year
0,Robert Arneson,American,Male,1930.0,1992.0
1,Doroteo Arnaiz,Spanish,Male,1936.0,NaN
2,Bill Arnold,American,Male,1941.0,NaN
3,Charles Arnoldi,American,Male,1946.0,NaN
4,Per Arnoldi,Danish,Male,1941.0,NaN


Отследивание скрытых пропусков с последующей заменой на NaN:

In [8]:
def tranform_hidden_missed(arg):
    hidden_missed_tags = ["unknown", "untitl", "not assigned", "dimensions var"]
    if type(arg) is str:
        for tag in hidden_missed_tags:
            if tag in arg.lower():
                return np.NaN
    return arg

for column in artw_df.columns:
    artw_df[column] = artw_df[column].apply(tranform_hidden_missed)

for column in artists_df.columns:
    artists_df[column] = artists_df[column].apply(tranform_hidden_missed)

Информация о названии и авторе некоторых произведениях отсутствует впринципе, поэтому все пропуски в столбце "title" требуется заменить на "Untitled", а все пропуски в столбце "name" на "Unknown artist", для реливантного вывода на front-end

In [9]:
artw_df.loc[artw_df.title.isnull(), 'title'] = 'Untitled'
artw_df.loc[artw_df.name.isnull(), 'name'] = 'Unknown artist'

Объединение таблиц по признаку "name":

In [10]:
arts_df = pd.merge(artw_df, artists_df, how='left', on='name')
arts_df.drop_duplicates(inplace=True)
arts_df.head()

,title,name,date,medium,dimensions,department,classification,diameter_cm,circumference_cm,height_cm,length_cm,width_cm,depth_cm,weight_kg,duration_s,nationality,gender,birth_year,death_year
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",Architecture & Design,Architecture,NaN,NaN,48.6000,NaN,168.9000,NaN,NaN,NaN,Austrian,Male,1841.0,1918.0
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",Architecture & Design,Architecture,NaN,NaN,40.6401,NaN,29.8451,NaN,NaN,NaN,French,Male,1944.0,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",Architecture & Design,Architecture,NaN,NaN,34.3000,NaN,31.8000,NaN,NaN,NaN,Austrian,Male,1876.0,1957.0
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",Architecture & Design,Architecture,NaN,NaN,50.8000,NaN,50.8000,NaN,NaN,NaN,NaN,Male,1944.0,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,1903,"Graphite, color pencil, ink, and gouache on tr...","15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",Architecture & Design,Architecture,NaN,NaN,38.4000,NaN,19.1000,NaN,NaN,NaN,Austrian,Male,1876.0,1957.0


Приобразование признаков размеров и признака "dimensions" в новыый признак "dims_list", представляющий из себя список уникальных значений размерности в сантиметрах (в последствии будет использован для категоризации размеров):

In [11]:
def transform_dimentions(sizes: pd.Series) -> list:
    dims = str(sizes.iloc[-1])
    other_sizes = sizes[:-1]

    digit_pattern = r"\d+\.\d+"
    match_cm_dims = re.findall(
        f"\\({digit_pattern}\\s+x\\s+{digit_pattern}\\s+cm\\s*\\)", dims)
    
    dims_set = set(other_sizes[~other_sizes.isnull()])

    if match_cm_dims:
        match_digit = re.findall(digit_pattern, match_cm_dims[0])
        dims_set.update(set(map(float, match_digit)))
    
    return [*dims_set] if dims_set else np.NaN

arts_df['dims_list'] = arts_df[['diameter_cm', 'circumference_cm', 
                               'height_cm', 'length_cm', 
                               'width_cm', 'depth_cm', 
                               'dimensions']].apply(transform_dimentions, axis=1)

Категоризация размеров произведений искусства по правилу:
1. Если по всем параметрам рамера значения меньше 30 см, категория: "small"
2. Если хотябы по одному параметру размера значение больше 100 см, категория "big"
3. Остальное "medium" 

In [12]:
def get_size_category(sizes: list) -> str:
    if sizes is np.NaN:
        return sizes
    sizes = np.array(sizes)

    if np.all(sizes < 30):
        return "small"
    elif np.any(sizes > 100):
        return "big"
    else:
        return "medium"

arts_df["size_category"] = arts_df.dims_list.apply(get_size_category)

Преобразование признака "date" в новый признак "release_date":

In [13]:
def tranform_date(date):
    if date is np.NaN:
        return date
    match_full_date = re.findall(r"\d{4}", date)
    match_part_date = re.search(r"-\d{2}", date)
    match_century = re.search(r"\dth century C.E.", date)

    if match_part_date and match_full_date:
        return int(match_full_date[0][:2] + match_part_date[0][1:])
    elif match_full_date:
        return int(match_full_date[-1])
    elif match_century:
        return int(match_century[0][0] + '00')
    else:
        return np.NaN

arts_df['release_date'] = arts_df.date.apply(tranform_date)

В колонке "release_date" много пропушенных значений. Для лучших показателей модели значения восстанавливаются brute force методом:

Гипотеза: деятели искусства создают большинство своих работ в середине жизни

Если есть информация о годе рождения автора произведения, то к этому значению прибавляется половина от медианного значения продолжительности жизни по выборке (значения грубые, но для модели это лучше, чем пропуски)

In [14]:
MEDIAN_HALH_LIFE = round((arts_df.death_year - arts_df.birth_year).median() / 2)

def release_date_recovery(years: pd.Series) -> pd.Series:
    birth, release = years
    if np.isnan(release) and not np.isnan(birth):
        release = birth + MEDIAN_HALH_LIFE
    # если огрублённое восстановленное значение привышает настоящий год (2023),
    # считаем что оно выпущено в этом году
    return 2023 if release > 2023 else release

    
arts_df.release_date = arts_df[['birth_year', 'release_date']].apply(release_date_recovery, axis=1)

Некоторые значения признака "gender" записаны с большой буквы, что создаёт больше 2 категорий.

Данная ошибка исправляется следующем преобразованием:

In [15]:
arts_df.gender = arts_df.gender.apply(lambda x: x.lower() if type(x) is str else x)

Некоторые произведения искусства не имеют физического воплощения (фильм, видео, дизайн, софт), но измеряются по длительности.

Это отражено в признаке "duration_s", которая приобразовывается в категориальный признак "duration_category", имеющий категории:

1. "short film" (меньше 52 минут)
2. "full length film" (52 минуты и больше)

In [16]:
def get_duration_category(duration: float) -> str:
    if np.isnan(duration):
        return duration
    
    duration_min = duration / 60
    if duration_min < 52:
        return "short film"
    else:
        return "full length film"

arts_df['duration_category'] = arts_df.duration_s.apply(get_duration_category)

В датасете присутствуют предметы искусства, которые совпадают друг с другом по всем признакам, кроме размеров (в основном это одни и те же фотографии или постеры, распечатанные в разных размерах).

Так как размер теперь категоризирован, эти признаки можно считать дубликатами.

Удаление лишних колонок, и удаление дубликатов:

In [17]:
dimentions_columns = ['diameter_cm', 'circumference_cm', 'height_cm', 'dimensions', 'length_cm', 
                      'width_cm', 'depth_cm', 'weight_kg', 'duration_s', 'dims_list', 'date']

arts_df.drop(dimentions_columns, axis=1, inplace=True)
arts_df.drop_duplicates(inplace=True)

Реорганизация датасета в нужном порядке и добавление признака "artwork_id":

In [18]:
# обновление индекса (после очистки и удаления дубликатов индексация некорректна)
arts_df['artwork_id'] = arts_df.index
columns_order = ['artwork_id', 'title', 'medium', 'department', 'classification', 
                 'size_category', 'duration_category', 'release_date', 
                 'name', 'nationality', 'gender', 'birth_year', 'death_year']
arts_df = arts_df[columns_order]
display(arts_df.head())

,artwork_id,title,medium,department,classification,size_category,duration_category,release_date,name,nationality,gender,birth_year,death_year
0,0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Ink and cut-and-pasted painted pages on paper,Architecture & Design,Architecture,big,NaN,1896.0,Otto Wagner,Austrian,male,1841.0,1918.0
1,1,"City of Music, National Superior Conservatory ...",Paint and colored pencil on print,Architecture & Design,Architecture,medium,NaN,1987.0,Christian de Portzamparc,French,male,1944.0,NaN
2,2,"Villa near Vienna Project, Outside Vienna, Aus...","Graphite, pen, color pencil, ink, and gouache ...",Architecture & Design,Architecture,medium,NaN,1903.0,Emil Hoppe,Austrian,male,1876.0,1957.0
3,3,"The Manhattan Transcripts Project, New York, N...",Photographic reproduction with colored synthet...,Architecture & Design,Architecture,medium,NaN,1980.0,Bernard Tschumi,NaN,male,1944.0,NaN
4,4,"Villa, project, outside Vienna, Austria, Exter...","Graphite, color pencil, ink, and gouache on tr...",Architecture & Design,Architecture,medium,NaN,1903.0,Emil Hoppe,Austrian,male,1876.0,1957.0


Так как в колонках с датами есть пропуски, они типизуруются, как float, что добавляет лишний разряд (лишний 0 после точки)

Для NLP модели все численные значения будут приобразованный в слова, следовательно чтобы избежать появления лишнего разряда и некорректной конвертации, значения типизированы к формату "str":

In [19]:
def delete_point(digit: float) -> str:
    if not np.isnan(digit):
        digit = int(digit) if int(digit) == digit else digit
        return str(digit)
    return digit

dates = ['birth_year', 'death_year', 'release_date']

for col in dates:
    arts_df[col] = arts_df[col].apply(delete_point)

На данном этапе часть датасета для front-end готова

Визуализация общей информации о преобразованном датасете для сравнения:

In [20]:
arts_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109834 entries, 0 to 133731
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   artwork_id         109834 non-null  int64 
 1   title              109834 non-null  object
 2   medium             98846 non-null   object
 3   department         109834 non-null  object
 4   classification     108932 non-null  object
 5   size_category      96108 non-null   object
 6   duration_category  3059 non-null    object
 7   release_date       108555 non-null  object
 8   name               109834 non-null  object
 9   nationality        100067 non-null  object
 10  gender             96821 non-null   object
 11  birth_year         95943 non-null   object
 12  death_year         65384 non-null   object
dtypes: int64(1), object(12)
memory usage: 11.7+ MB


Отсюда видно, что количество объектов уменьшилось благодаря категоризации и удалению дубликатов

### Но необходимо сделать ещё один признак для NLP модели

Требования:
1. Признак представлен в виде объединённого текста всех признаков
2. Текст в нижнем регистре, очищен от лишних пробелов, символов и знаков припенания
3. Все числа преобразованны в слова
4. Текст очищен от стоп слов и лемматизирован

In [21]:
# функция преобразования чисел в слова
def digit_to_words(digit: float) -> str:
    p = inflect.engine()
    text = p.number_to_words(digit)
    return text

# функция очистки текста от лишних символов и пробелов
def clear_text(text) -> str:
    if type(text) is not str:
        return text

    digit_pattern = r"\d+[\s\.]?\d*"
    symbol_pattern = r"[\W_]"

    while True:
        match_digit = re.search(digit_pattern, text)
        if not match_digit:
            break
        transformed_digit = digit_to_words(match_digit[0])
        text = re.sub(digit_pattern, f' {transformed_digit} ', text, count=1)

    text = re.sub(symbol_pattern, ' ', text)
    text = re.sub(r"\s+", ' ', text)
    return text.lower().strip()

Основное приобразование в признак "feature_joined_text" для NLP модели (процесс может занимать более 20 минут):

In [22]:
# копирование датасета для безопасности преобразований
arts_df_to_model = arts_df.copy()

# выделение колонок, информативных для модели 
columns_to_model = ['title', 'name', 'release_date', 'medium', 'department', 'classification', 
                    'size_category', 'duration_category', 'nationality', 'gender']

# инициализация nlp пакета для английского языка
nlp = spacy.load('en_core_web_lg')

# результирующая функция преобразования текста
def get_clear_text(sample: pd.Series) -> str:
    # первичная очистка
    clear_sample = sample.apply(clear_text)
    features_texts = [*clear_sample[~clear_sample.isnull()]]

    # удаление стоп слов и лемматизация
    doc = nlp(' '.join(features_texts))
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(tokens)

arts_df_to_model["feature_joined_text"] = arts_df_to_model[columns_to_model] \
                                          .apply(get_clear_text, axis=1)

Демонстрация результирующего признака "feature_joined_text":

In [23]:
display(*arts_df_to_model.feature_joined_text.head().values)

'ferdinandsbrücke project vienna austria elevation preliminary version otto wagner thousand ninety ink cut paste paint page paper architecture design architecture big austrian male'

'city music national superior conservatory music dance paris france view interior courtyard christian de portzamparc thousand eighty seven paint colored pencil print architecture design architecture medium french male'

'villa near vienna project outside vienna austria elevation emil hoppe thousand graphite pen color pencil ink gouache trace paper architecture design architecture medium austrian male'

'manhattan transcript project new york new york introductory panel episode park bernard tschumi thousand eighty photographic reproduction colored synthetic laminate architecture design architecture medium male'

'villa project outside vienna austria exterior perspective emil hoppe thousand graphite color pencil ink gouache trace paper architecture design architecture medium austrian male'

Сохранение готового датасета в CSV-файл:

In [24]:
arts_df_to_model.to_csv("arts_data_model_and_front.csv", encoding='utf-8', index=False)